In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import chromedriver_binary
import string

pd.options.display.float_format = '{:.0f}'.format

In [2]:
ticker = input("Input the ticker of the company you'd like to see the financials of: ")

is_link = f'https://finance.yahoo.com/quote/{ticker}/financials?p={ticker}'
bs_link = f'https://finance.yahoo.com/quote/{ticker}/balance-sheet?p={ticker}'
cf_link = f'https://finance.yahoo.com/quote/{ticker}/cash-flow?p={ticker}'

Input the ticker of the company you'd like to see the financials of: AAPL


In [54]:
def yahoo_financial_statements():
    
    ticker = input("Input the ticker of the company you'd like to see the financials of: ")

    is_link = f'https://finance.yahoo.com/quote/{ticker}/financials?p={ticker}'
    bs_link = f'https://finance.yahoo.com/quote/{ticker}/balance-sheet?p={ticker}'
    cf_link = f'https://finance.yahoo.com/quote/{ticker}/cash-flow?p={ticker}'
    
    statements_list = [is_link,bs_link,cf_link]
    
    headers = []
    temp_list = []
    label_list = []
    final = []
    index = 0
    
    df_lists = list()
    driver = webdriver.Chrome()
    
    for link in statements_list:
        
        
        driver.get(link)
        html = driver.execute_script('return document.body.innerHTML;')
        soup = BeautifulSoup(html,'lxml')
        
        features = soup.find_all('div', class_='D(tbr)')
        
        #create headers
        for item in features[0].find_all('div', class_='D(ib)'):
            headers.append(item.text)
            
        #statement contents
        while index <= len(features)-1:
            #filter for each line of the statement
            temp = features[index].find_all('div', class_='D(tbc)')
            for line in temp:
                #each item added to a temp list
                temp_list.append(line.text)
            #temp_list added to final list
            final.append(temp_list)
            #clear temp_list
            temp_list = []
            index+=1
        
        df = pd.DataFrame(final[1:])
        df.columns = headers
        df.index = final[1]
        
        #function to make all values numerical
        def convert_to_numeric(column):

            first_col = [i.replace(',','') for i in column]
            second_col = [i.replace('-','') for i in first_col]
            final_col = pd.to_numeric(second_col)

            return final_col
        
        for column in headers[1:]:
            df[column] = convert_to_numeric(df[column])
        
        final_df = df.fillna('-')
        df_lists.append(final_df)
        
        #reset all lists
        headers = []
        temp_list = []
        label_list = []
        final = []
        index = 0

    return df_lists
        

In [55]:
financials = yahoo_financial_statements()

Input the ticker of the company you'd like to see the financials of: aapl


ValueError: Length mismatch: Expected axis has 22 elements, new values have 6 elements

In [46]:
income_statement = financials[0]
balance_sheet = financials[1]
cash_flow_statement = financials[2]

In [66]:
income_statement.index = income_statement[['Breakdown']]

In [71]:
income_statement

,Breakdown,ttm,9/30/2019,9/30/2018,9/30/2017,9/30/2016
"(Cost of Revenue,)",Cost of Revenue,166105000,161782000,163756000,141048000,131376000
"(Gross Profit,)",Gross Profit,101578000,98392000,101839000,88186000,84263000
"(Operating Expenses,)",Operating Expenses,-,-,-,-,-
"(Research Development,)",Research Development,16766000,16217000,14236000,11581000,10045000
"(Selling General and Administrative,)",Selling General and Administrative,18659000,18245000,16705000,15261000,14194000
"(Total Operating Expenses,)",Total Operating Expenses,35425000,34462000,30941000,26842000,24239000
"(Operating Income or Loss,)",Operating Income or Loss,66153000,63930000,70898000,61344000,60024000
"(Interest Expense,)",Interest Expense,3471000,3576000,3240000,2323000,1456000
"(Total Other Income/Expenses Net,)",Total Other Income/Expenses Net,368000,422000,441000,133000,1195000
"(Income Before Tax,)",Income Before Tax,67749000,65737000,72903000,64089000,61372000


In [70]:
income_statement.iloc[0,1]

166105000.0

In [77]:
income_statement.iloc[0][2]

161782000.0

In [81]:
income_statement[['Breakdown']].iloc[0]

Breakdown    Cost of Revenue
Name: (Cost of Revenue,), dtype: object